In [0]:
# Install the tabulate module (if running in an environment like Databricks or Jupyter)
# %pip install tabulate

import pandas as pd
from datetime import datetime
from tabulate import tabulate # For pretty printing tables

# --- CORRECT SAMPLE DATA (from LeetCode Problem 262 Example 1) ---

# Users table data as per LeetCode Example 1
users = pd.DataFrame({
    'users_id': [1, 2, 3, 4, 10, 11, 12, 13],
    'banned': ['No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No'],
    'role': ['client', 'client', 'client', 'client', 'driver', 'driver', 'driver', 'driver']
})

# Trips table data as per LeetCode Example 1
trips = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'client_id': [1, 2, 3, 4, 1, 2, 3, 2, 3, 4],
    'driver_id': [10, 11, 12, 13, 10, 11, 12, 12, 10, 13],
    'city_id': [1, 1, 6, 6, 1, 6, 6, 12, 12, 12],
    'status': ['completed', 'cancelled_by_driver', 'completed', 'cancelled_by_client', 'completed', 'completed', 'completed', 'completed', 'completed', 'cancelled_by_driver'],
    'request_at': ['2013-10-01', '2013-10-01', '2013-10-01', '2013-10-02', '2013-10-02', '2013-10-02', '2013-10-03', '2013-10-03', '2013-10-03', '2013-10-04']
})

# Convert 'request_at' column to datetime objects for proper filtering
trips['request_at'] = pd.to_datetime(trips['request_at'])

# --- Solution Logic ---

# 1. Merge users data twice to get banned status for both client and driver
#    This is crucial as 'client_id' and 'driver_id' are both 'users_id' from the Users table.

# Merge for client banned status
df = trips.merge(
    users[['users_id', 'banned']].rename(columns={'users_id': 'client_id', 'banned': 'client_banned'}),
    on='client_id',
    how='left'
)

# Merge for driver banned status
df = df.merge(
    users[['users_id', 'banned']].rename(columns={'users_id': 'driver_id', 'banned': 'driver_banned'}),
    on='driver_id',
    how='left'
)

# 2. Filter out trips where either the client or the driver is banned
df_filtered = df[(df['client_banned'] == 'No') & (df['driver_banned'] == 'No')]

# 3. Filter for the specified date range
start_date = datetime.strptime("2013-10-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2013-10-03", "%Y-%m-%d").date()

# Convert DataFrame's request_at to date objects for comparison
df_filtered['request_at_date'] = df_filtered['request_at'].dt.date
df_filtered = df_filtered[(df_filtered['request_at_date'] >= start_date) & (df_filtered['request_at_date'] <= end_date)]

# 4. Identify cancelled trips
df_filtered['is_cancelled'] = df_filtered['status'].apply(lambda x: x.startswith('cancelled'))

# 5. Group by request_at (date) and calculate total and cancelled requests
summary = df_filtered.groupby('request_at_date').agg(
    total_requests=('id', 'count'),
    cancelled_requests=('is_cancelled', 'sum')
).reset_index()

# 6. Ensure all days in the range are present, even if they had no valid trips
#    Create a full date range and left merge with the summary
full_date_range = pd.DataFrame(pd.date_range(start=start_date, end=end_date), columns=['request_at_date'])
full_date_range['request_at_date'] = full_date_range['request_at_date'].dt.date

summary = full_date_range.merge(summary, on='request_at_date', how='left')

# Fill NaNs with 0 for counts, and 0.00 for cancellation rate
summary['total_requests'] = summary['total_requests'].fillna(0).astype(int)
summary['cancelled_requests'] = summary['cancelled_requests'].fillna(0).astype(int)

# 7. Calculate cancellation rate and round to two decimal places
#    Handle division by zero for days with no total requests
summary['Cancellation Rate'] = summary.apply(
    lambda row: round(row['cancelled_requests'] / row['total_requests'], 2) if row['total_requests'] > 0 else 0.00,
    axis=1
)

# 8. Format date column back to string 'YYYY-MM-DD'
summary['Day'] = summary['request_at_date'].astype(str)

# 9. Select and rename columns for final output
result = summary[['Day', 'Cancellation Rate']]

# --- Print Final Output in Table Format ---
print("Pandas Solution Output (using LeetCode example data):")
# Use tabulate to print the DataFrame as a GitHub-flavored Markdown table
print(tabulate(result, headers='keys', tablefmt='github'))